# AIC-25 Colab: Textual KIS (CLI flow)

This notebook clones the repo, downloads dataset assets from a CSV of links, wires the dataset into the backend, starts the API, and exports KIS CSVs using the CLI tools.

Note: TRAKE requires Sonic + transcripts/heading JSON ingestion and is not covered here.

In [ ]:
# Clone the repo into /content/aic-25
import os, shutil, subprocess
REPO_URL = "https://github.com/dnlqvu/hcm-AI-challenge-2024-main.git"
TARGET_DIR = "/content/aic-25"

if os.path.exists(TARGET_DIR):
    shutil.rmtree(TARGET_DIR)
print('Cloning', REPO_URL, '->', TARGET_DIR)
subprocess.run(['git', 'clone', REPO_URL, TARGET_DIR], check=True)

# List top-level files to confirm
print(os.listdir(TARGET_DIR))


In [ ]:
# Install backend dependencies
%cd /content/aic-25/aic-24-BE
!python -m pip install --upgrade pip
!pip install -r requirements.txt


In [ ]:
# Upload AIC_2025_dataset_download_link.csv (or edit CSV_PATH to an accessible path)
from google.colab import files
uploaded = files.upload()  # choose AIC_2025_dataset_download_link.csv
CSV_PATH = next(iter(uploaded))
print('Using CSV:', CSV_PATH)


In [ ]:
# Download dataset assets to example_dataset/ and extract
%cd /content/aic-25
!python tools/aic_cli.py download-dataset --csv $CSV_PATH --outdir example_dataset --extract
!ls -la example_dataset | head -n 50


In [ ]:
# Wire dataset into backend: copy, rename keyframes, pack features, build model
%cd /content/aic-25
!python tools/aic_cli.py setup-example --example-dir example_dataset
!ls -la aic-24-BE/data/video_frames | head -n 20
!ls -la aic-24-BE/data/clip_features | head -n 20
!cat aic-24-BE/.env


In [ ]:
# Start backend API in the background on port 8000
%cd /content/aic-25
!python tools/aic_cli.py serve --port 8000 --run &
import time; time.sleep(5)
!python - << 'PY'
import requests
try:
    r = requests.get('http://localhost:8000/docs', timeout=10)
    print('Backend reachable:', r.status_code)
except Exception as e:
    print('Backend not reachable yet:', e)
PY


In [ ]:
# Create a demo KIS query
%cd /content/aic-25
!mkdir -p queries_demo
query_text = 'tin tức thời sự'  # edit your KIS query here
open('queries_demo/query-1-kis.txt', 'w', encoding='utf-8').write(query_text)
!sed -n '1,5p' queries_demo/query-1-kis.txt


In [ ]:
# Export KIS CSVs to submission/
%cd /content/aic-25
!python tools/aic_cli.py export --queries ./queries_demo --api http://localhost:8000 --outdir submission
!echo 'Generated files:' && ls -la submission
!echo 'Preview:' && head -n 5 submission/query-1-kis.csv


In [ ]:
# (Optional) Zip for Codabench and download
%cd /content/aic-25
!python tools/aic_cli.py zip-submission --outdir submission --name aic25_submission.zip
from google.colab import files as colab_files
colab_files.download('aic25_submission.zip')
